In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
#Parámetros de búsqueda
url_basica="https://www.medicamentosplm.com/Home/Medicamento/"
#Letras de A a Z
letras=[chr(i) for i in range(65,91)]
#Números del uno al 36
numeros=[str(i) for i in range(1,37)]
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}

In [3]:
print(letras)
print(numeros)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36']


In [4]:
#Crear las urls de consulta
urls=[]
for letra in letras:
    for numero in numeros:
        urls.append(url_basica+letra+"/"+numero)
len(urls)

936

In [5]:
#Funciones
def clean_text(text):
    """FGunnción para limpiar el texto scrapeado."""
    return text.strip().replace("\n", "").replace("\t", "")

def extract_data_from_elements(elements):
    """Extrae el tetxto."""
    return [clean_text(element.text) for element in elements]


def extract_data_from_url(url):
    """Función para generar el requests y obtener la info."""
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    # Obtner tabla
    table = soup.find("div", {"id":"col-content-catalogs"})

    # Diccionarios de clases
    class_dict = {
        "nombre_producto": "nombrePrducto"
    }

    id_dict = {
        "sustancia": lambda x: x and "prescription-substance" in x,
        "forma_farm": lambda x: x and "prescription-pharmaForm" in x,
        "lab": lambda x: x and "prescription-division" in x,
        "presentacion": lambda x: x and "prescription-presentation" in x
    }

    # Extraer datos
    data = {}

    for key, value in class_dict.items():
        try:
            elements = table.find_all("div", {"class": value})
            data[key] = extract_data_from_elements(elements)
        except:
            data[key] = None

    for key, value in id_dict.items():
        try:
            elements = table.find_all("div", {"id": value})
            data[key] = extract_data_from_elements(elements)
        except:
            data[key] = None

    return data

In [6]:
nombre_producto = []
sustancia = []
forma_farm = []
lab = []
presentacion = []

# Iterar sobre las urls
for url in urls:
    try:
        data = extract_data_from_url(url)
        nombre_producto.extend(data["nombre_producto"])
        sustancia.extend(data["sustancia"])
        forma_farm.extend(data["forma_farm"])
        lab.extend(data["lab"])
        presentacion.extend([data["presentacion"][i] for i in range(0,len(data["presentacion"]),2)])
    except:
        data = None

In [7]:
#nombre_producto y sustancia a df
df=pd.DataFrame({"nombre_producto":nombre_producto,"sustancia":sustancia, "forma_farm":forma_farm, "lab":lab, "presentacion":presentacion})
df=df.apply(lambda x: x.astype(str).str.lower())
#Eliminar acentos en tdas las columnas
df=df.apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))
df

,nombre_producto,sustancia,forma_farm,lab,presentacion
0,a.f. valdecasas,vitamina b9 (acido folico),tabletas,valdecasas,"1 frasco(s) , 20 tabletas , 1.0 miligramos"
1,a.f. valdecasas,vitamina b9 (acido folico),tabletas,valdecasas,"1 frasco(s) , 20 tabletas , 5.0 miligramos"
2,a.f. valdecasas,vitamina b9 (acido folico),tabletas,valdecasas,"1 frasco(s) , 30 tabletas , 0.4 miligramos"
3,a.f. valdecasas,vitamina b9 (acido folico),tabletas,valdecasas,"1 frasco(s) , 375 tabletas , 5.0 miligramos"
4,a.f. valdecasas,vitamina b9 (acido folico),tabletas,valdecasas,"1 frasco(s) , 375 tabletas , 0.4 miligramos"
...,...,...,...,...,...
17585,zyxem,levocetirizina,solucion gotas,chinoin,"1 caja, 1 frasco gotero, 15 ml, 5 mg/ml"
17586,zyxem,levocetirizina,solucion gotas,chinoin,"1 caja, 1 frasco con gotero, 20 ml, 5 mg/ml"
17587,zyxem,levocetirizina,tabletas,chinoin,"1 caja, 10 tabletas, 5 miligramos"
17588,zyxem,levocetirizina,tabletas,chinoin,"1 caja, 20 tabletas, 5 miligramos"


In [8]:
#Salvar a csv
df.to_csv("meds_plm.csv", index=False, encoding="utf-8")